## 6.7.1 扩展内置类型

In [1]:
# python的所有内置类型都支持直接创建子类，比如下面给list创建一个子类
class Mylist(list):
    def __getitem__(self, offset):
        print('indexing %s at %s' % (self,offset))
        return list.__getitem__(self, offset-1)

In [2]:
# 这样我们就可以从1开始计算索引了
print(list('abc'))
x = Mylist('abc')
print(x)
print(x[1])
print(x[3])
x.append('spam')
print(x)
x.reverse()
print(x)

['a', 'b', 'c']
['a', 'b', 'c']
indexing ['a', 'b', 'c'] at 1
a
indexing ['a', 'b', 'c'] at 3
c
['a', 'b', 'c', 'spam']
['spam', 'c', 'b', 'a']


## 6.7.2 新式类模型

In [ ]:
# python2.2 之后的类统一被称为新式类，这里就不讲具体区别了

## 6.7.3 新式类变化

In [6]:
# 当我们定义内置运算的时候，不会路由到getattr
class c(object):
    data = 'spam'
    def __getattr__(self, name):
        print('getattr:'+name)
        return getattr(self.data, name)
    def __getitem__(self, i):
        print('getitem:'+str(i))
        return self.data[i]
    def __add__(self, other):
        print('add:'+other)
        return getattr(self.data, '__add__')(other)

In [7]:
x = c()
print(x.upper())
print(x[1])
print(x.__getitem__(1))
print(x+'eggs')

getattr:upper
SPAM
getitem:1
p
getitem:1
p
add:eggs
spameggs


In [8]:
# 类就是类型，类型就是类
class c(object): pass
i =c()
print(type(i), i.__class__)
print(type(c), c.__class__)

<class '__main__.c'> <class '__main__.c'>
<class 'type'> <class 'type'>


In [10]:
# 所有的对象都派生自object
class c: pass
x = c()
print(type(x), type(c))
print(isinstance(x, object))
print(isinstance(c, object))

<class '__main__.c'> <class 'type'>
True
True


In [11]:
class A: attr=1
class B(A): pass
class C(A): attr = 2
class D(B,C): pass
x = D()
print(x.attr)

2


In [12]:
# 为了避免疑惑，我们可以直接手动指定
class A: attr=1
class B(A): pass
class C(A): attr = 2
class D(B,C): attr = C.attr
x = D()
print(x.attr)

2


## 6.7.4 新式类扩展

### 6.7.4.1 slot

![](.6_images/d7e37f58.png)

In [13]:
# 我们可以通过slot来让python对属性进行检查
class limiter(object):
    __slots__ = ['age','name','job']
x = limiter()
# 因为我们没有声明变量，所以直接访问会出错
print(x.age)

AttributeError: age

In [14]:
x.age = 40
print(x.age)

40


In [15]:
# 如果不是solt里面会报其他错误
x.ape = 1000

AttributeError: 'limiter' object has no attribute 'ape'

In [16]:
class c:
    __slots__ = ['a','b']
x = c()
x.a = 1
print(x.a)
# 使用了slots会影响dict
print(x.__dict__)

1


AttributeError: 'c' object has no attribute '__dict__'

In [18]:
# 不过我们可以使用getattr方法
print(getattr(x, 'a'))
# 手动设置属性
setattr(x,'b',2)
print(x.b)
print('a' in dir(x))

1
2
True


In [1]:
# 当然也可以把dict放入slots中
class D:
    __slots__ = ['a','b','__dict__']
    c = 3
    def __init__(self):
        self.d = 4
x = D()
print(x.d)
print(x.c)
print(x.a)

4
3


AttributeError: a

In [3]:
x.a=1
# 此时我们可以使用dict
print(x.__dict__)
print(x.__slots__)
# 但是getattr依旧可以正常获取到所有的值
print(getattr(x,'a'),getattr(x,'c'),getattr(x,'d'))

{'d': 4}
['a', 'b', '__dict__']
1 3 4


In [4]:
# 父类中有多个slots列表的情况
class E:
    __slots__ = ['c','d']
class D(E):
    __slots__ = ['a','__dict__']
x = D()
x.a = 1;x.b=2;x.c=3
print(x.a,x.c)

1 3


In [6]:
# 下面我们看一下对应的slots，其实我们无法获取搞层次的slot列表
print(x.__slots__)
print(D.__slots__)
print(E.__slots__)
# 但是如果检查dict会发现确实会被继承
print(x.__dict__)

['a', '__dict__']
['a', '__dict__']
['c', 'd']
{'b': 2}


In [7]:
# 实际上，我们也可以通过dir直接把所有的属性全部列出来
print(dir(x))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', 'a', 'b', 'c', 'd']


In [ ]:
# 一般情况slot不推荐使用，虽然可以减少内存空间，但是会导致我们程序变得更加复杂并且破坏我们代码的结构

### 6.7.4.2 property属性访问器

In [8]:
# 经典类会使用getattr来获取属性
class operators:
    def __getattr__(self, name):
        if name == 'age':
            return 40
        else:
            raise AttributeError(name)
x = operators()
print(x.age)
print(x.name)

40


AttributeError: name

In [9]:
# 下面我们改写一下
class properties(object):
    def getage(self):
        return 40
    age = property(getage,None,None,None)
x = properties()
print(x.age)
print(x.name)

40


AttributeError: 'properties' object has no attribute 'name'

In [10]:
# 当我们要修改属性的时候，可以这样
class properties(object):
    def getage(self):
        return 40
    def setage(self, value):
        print('set age:%d' % value)
        self._age = value
    age = property(getage,setage,None,None)
x = properties()
print(x.age)
x.age = 42
print(x.age)
print(x._age)
x.job = 'trainer'
print(x.job)

40
set age:42
40
42
trainer


### 6.7.4.3 getattribute 和描述符：属性工具

![](.6_images/8e861d8f.png)

In [11]:
class AgeDesc(object):
    def __get__(self, instance, owner): return 40
    def __set__(self, instance, value): instance._age = value
class descriptors(object):
    age = AgeDesc()
x = descriptors()
print(x.age)
x.age = 42
print(x._age)

40
42


## 6.7.5 静态方法和类方法

In [15]:
# 我们可以给类添加属性和类方法，这样所有实例之间都可以共享
class Spam:
    numInstances = 0
    def __init__(self):
        Spam.numInstances = Spam.numInstances + 1
    def printNumInstance():
        print('number of instance create:', Spam.numInstances)

In [16]:
a = Spam()
b = Spam()
c = Spam()
Spam.printNumInstance()

number of instance create: 3


In [17]:
# 如果对实例化的对象直接打印则会报错
a.printNumInstance()

TypeError: printNumInstance() takes 0 positional arguments but 1 was given

In [18]:
# 如果我们设置类方法添加self也是可以的，但是需要我们传入初始化好的对象
class Spam:
    numInstances = 0
    def __init__(self):
        Spam.numInstances = Spam.numInstances + 1
    def printNumInstance(self):
        print('number of instance create:', Spam.numInstances)
a = Spam()
b = Spam()
c = Spam()
a.printNumInstance()
Spam.printNumInstance(a)
Spam().printNumInstance()

number of instance create: 3
number of instance create: 3
number of instance create: 4


In [19]:
# 当然，我们也可以使用静态方法
class Methods:
    def imeth(self,x):
        print([self,x])
    def smeth(x):
        print([x])
    def cmeth(cls,x):
        print([cls,x])
    # 下面这个就表示那些是静态方法
    smeth = staticmethod(smeth)
    cmeth = staticmethod(cmeth)

In [21]:
obj = Methods()
obj.imeth(1)
Methods.imeth(obj,2)
Methods.smeth(3)
obj.smeth(3)

[<__main__.Methods object at 0x0000017BEE587788>, 1]
[<__main__.Methods object at 0x0000017BEE587788>, 2]
[3]
[3]


## 6.7.6 装饰器和元素

In [25]:
# 我们可以直接用装饰器来表示静态函数
class Methods:
    def imeth(self,x):
        print([self,x])
    # 静态方法装饰器
    @staticmethod
    def smeth(x):
        print([x])
    # 类装饰器
    @classmethod
    def cmeth(cls,x):
        print([cls,x])
    # 直使用properties装饰器来表示我们的这个属性
    @property
    def name(self):
        return 'Bob' + self.__class__.__name__

In [26]:
obj = Methods()
obj.imeth(2)
obj.cmeth(3)
print(obj.name)

[<__main__.Methods object at 0x0000017BEE53C688>, 2]
[<class '__main__.Methods'>, 3]
BobMethods


In [27]:
# 我们也可以自己定义装饰器
class tracer:
    def __init__(self, func):
        self.calls = 0
        self.func = func
    def __call__(self, *args, **kwargs):
        self.calls += 1
        print("call %s to %s" % (self.calls,self.func.__name__))
        return self.func(*args)
@tracer
def spam(a,b,c):
    return a+b+c
print(spam(1,2,3))
print(spam('a','b','c'))

call 1 to spam
6
call 2 to spam
abc


In [28]:
# 自定义的装饰器还可以用于类方法
def tracer(func):
    def oncall(*args):
        oncall.calls += 1
        print("call %s to %s" % (oncall.calls, func.__name__))
        return func(*args)
    oncall.calls = 0
    return oncall

class C:
    @tracer
    def spam(self,a,b,c): return a+b+c

x = C()
print(x.spam(1,2,3))
print(x.spam('a','b','c'))

call 1 to spam
6
call 2 to spam
abc


In [29]:
# 这些东西第8章会深入讲解一下

## 6.7.7 supper内置函数

In [30]:
# 传统调用方式如下
class C:
    def act(self):
        print('spam')
class D(C):
    def act(self):
        C.act(self)
        print('eggs')
x = D()
x.act()

spam
eggs


In [31]:
# 我们也可以使用supper来调用
class C:
    def act(self):
        print('spam')
class D(C):
    def act(self):
        super().act()
        print('eggs')
x = D()
x.act()

spam
eggs


In [32]:
# supper方法不能直接调用
super()

RuntimeError: super(): no arguments

In [33]:
class E(C):
    def method(self):
        proxy = super()
        print(proxy)
        proxy.act()
E().method()

<super: <class 'E'>, <E object>>
spam


In [35]:
# super只能单继承
class A:
    def act(self): print('A')
class B:
    def act(self): print('B')
class C(A):
    def act(self): super().act()
x = C()
x.act()

A


In [36]:
# 如果我们这样操作的话，不会报错，会直接使用第一个
class C(A,B):
    def act(self):
        super().act()
x = C()
x.act()

A


In [37]:
# 我们可以换一下顺序，继承的关系就变了
class C(B,A):
    def act(self):
        super().act()
x = C()
x.act()

B


In [2]:
# 运算符重载有一定的局限性
class C:
    def __getitem__(self, ix):
        print('c index')
class D(C):
    def __getitem__(self, ix):
        print('d index')
        c.__getitem__(self, ix)
        # 这样调用没有问题
        super().__getitem__(ix)
        # 但是不能这样调用
        super()[ix]
x = C()
x[99]
x = D()
x[99]

c index
d index


NameError: name 'c' is not defined

## 6.7.8 类陷阱

In [5]:
# 类属性支持修改
class X:
    a = 1
i = X()
# 前面这块没啥问题
print(i.a)
print(X.a)

1
1


In [6]:
# 如果此时，我们取修改内容
X.a = 2
print(X.a)
print(i.a)
# 我们新建一个类，此时j也会被修改
j = X()
print(j.a)

2
2
2


In [7]:
# 修改可变类属性也可能会产生副作用
class C:
    shared = []
    def __init__(self):
        self.perobj = []
x = C()
y = C()
print(x.shared, y.perobj)

[] []


In [8]:
x.shared.append('spam')
x.perobj.append('spam')
print(x.shared,x.perobj)
# 这里我们没有修改y，但是y还是被修改了
print(y.shared, y.perobj)
print(C.shared)

['spam'] ['spam']
['spam'] []
['spam']
